In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [35]:
cameraman = "C:/Users/20092/OneDrive/Documents/Bhumit/cyptography/cameraman.png"
peppers = "C:/Users/20092/OneDrive/Documents/Bhumit/cyptography/peppers.png"

In [47]:
camera = Image.open(cameraman).convert('RGB')  # Color image
cameraman_array = np.array(camera)
pepper = Image.open(peppers).convert('RGB')  # Color image
peppers_array = np.array(pepper)

In [102]:
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')  # Color image
    image_array = np.array(image)
    return image_array

In [101]:
import os

def load_dataset(folder_path):
    """Load all images from the dataset folder."""
    image_paths = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')):
                image_paths.append(os.path.join(root, file))
    return image_paths


In [36]:
def plot_images(original_image, encrypted_image, decrypted_image, shape):
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))

    axes[0].imshow(original_image.reshape(shape))
    axes[0].set_title("Original Image")
    axes[0].axis('off')

    axes[1].imshow(encrypted_image.reshape(shape))
    axes[1].set_title("Encrypted Image")
    axes[1].axis('off')

    axes[2].imshow(decrypted_image.reshape(shape))
    axes[2].set_title("Decrypted Image")
    axes[2].axis('off')

    difference_image = np.abs(original_image - decrypted_image)
    axes[3].imshow(difference_image.reshape(shape))
    axes[3].set_title("Difference Image")
    axes[3].axis('off')

    plt.show()

In [82]:
import numpy as np
import random

def modify_single_pixel(image_array):
    """
    Modify a single pixel in the image by assigning it a random value.
    
    Parameters:
    image_array (numpy.ndarray): The input image as a NumPy array.
    
    Returns:
    numpy.ndarray: The modified image.
    tuple: The coordinates of the modified pixel.
    """
    # Get the dimensions of the image
    height, width, channels = image_array.shape

    # Randomly select a pixel (row, column)
    row = random.randint(0, height - 1)
    col = random.randint(0, width - 1)

    # Randomly assign a new value to the pixel
    random_value = np.random.randint(0, 256, size=channels, dtype=np.uint8)
    modified_image = image_array.copy()  # Create a copy to avoid altering the original image
    modified_image[row, col] = random_value

    return modified_image


In [38]:
def change_image(image_path):
    image = Image.open(image_path).convert('RGB')  # Color image
    image_array = np.array(image)
    modified_image_array = modify_single_pixel(image_array)
    return modified_image_array

In [98]:
import numpy as np
from PIL import Image

import wave
import struct


def compute_image_characteristics(image_array):
    avg_intensity = np.mean(image_array)  # Average pixel intensity
    variance = np.var(image_array)  # Variance of pixel intensity
    histogram, _ = np.histogram(image_array, bins=256, range=(0, 255))
    probabilities = histogram / np.sum(histogram)
    entropy = -np.sum(p * np.log2(p) for p in probabilities if p > 0)  # Entropy calculation
    return avg_intensity, variance, entropy

def determine_block_size(image_shape):
    print(image_shape)
    height, width = image_shape[:2]
    smaller_dim = min(height, width)
    if smaller_dim <200:
        return 1
    if smaller_dim < 512:
        return 16  # Small images
    elif smaller_dim <= 2048:
        return 64  # Medium images
    else:
        return 64 # Large images

def divide_into_blocks(image_array, block_size):
    height, width, channels = image_array.shape
    padded_height = (height + block_size - 1) // block_size * block_size
    padded_width = (width + block_size - 1) // block_size * block_size

    padded_image = np.zeros((padded_height, padded_width, channels), dtype=image_array.dtype)
    padded_image[:height, :width, :] = image_array

    blocks = []
    for i in range(0, padded_height, block_size):
        for j in range(0, padded_width, block_size):
            blocks.append(padded_image[i:i + block_size, j:j + block_size])
    return blocks, (height, width), (padded_height, padded_width)

def merge_blocks(blocks, original_shape, padded_shape, block_size):
    padded_height, padded_width = padded_shape
    merged_image = np.zeros((padded_height, padded_width, blocks[0].shape[2]), dtype=blocks[0].dtype)

    block_index = 0
    for i in range(0, padded_height, block_size):
        for j in range(0, padded_width, block_size):
            merged_image[i:i + block_size, j:j + block_size] = blocks[block_index]
            block_index += 1

    return merged_image[:original_shape[0], :original_shape[1], :]

def generate_chen_keys(size, avg_intensity, variance):
    x, y, z = avg_intensity / 255.0, variance / 255.0, 0.1  # Dynamic initialization
    a, b, c = 35 + avg_intensity % 10, 3 + variance % 5, 28  # Adjusted parameters
    keys = []
    for _ in range(size):
        dx = a * (y - x)
        dy = x * (c - z) - y
        dz = x * y - b * z
        x += dx * 0.01
        y += dy * 0.01
        z += dz * 0.01
        keys.append(x)
    return np.abs(np.array(keys)) % 256  # Normalize keys to range [0, 255]

def generate_lorenz_keys(size, avg_intensity, entropy):
    x, y, z = avg_intensity / 255.0, 0.5, entropy / 10.0  # Dynamic initialization
    sigma, rho, beta = 10 + entropy % 5, 28 + avg_intensity % 10, 2.667
    keys = []
    for _ in range(size):
        dx = sigma * (y - x)
        dy = x * (rho - z) - y
        dz = x * y - beta * z
        x += dx * 0.01
        y += dy * 0.01
        z += dz * 0.01
        keys.append(y)
    return np.abs(np.array(keys)) % 256

def generate_logistic_keys(size, variance, entropy):
    x = np.clip(variance / 255.0, 0.01, 0.99)  
    r = np.clip(3.99 - (entropy % 0.1), 3.57, 4.0)  
    keys = []
    for _ in range(size):
        x = r * x * (1 - x)  
        keys.append(x)
    return (np.array(keys) * 255).astype(np.uint8)

def generate_dynamic_sbox(size, hybrid_keys):
    chaotic_sequence = hybrid_keys[:size]
    return np.argsort(chaotic_sequence)

def encrypt_block_cbc(block, hybrid_keys, sbox, previous_block):
    flat_block = block.flatten()

    xor_block = flat_block ^ previous_block.flatten()

    permutation_indices = np.argsort(hybrid_keys[:len(flat_block)])
    permuted_block = xor_block[permutation_indices]
    substituted_block = sbox[permuted_block]

    return substituted_block.reshape(block.shape), substituted_block.flatten(), permutation_indices

def decrypt_block_cbc(block, previous_block, permutation_indices, sbox_inv):
    flat_block = block.flatten()

    substituted_block = np.array([sbox_inv[pixel] for pixel in flat_block])

    reverse_indices = np.argsort(permutation_indices)
    permuted_block = substituted_block[reverse_indices]

    decrypted_block = permuted_block ^ previous_block.flatten()

    return decrypted_block.reshape(block.shape)



def main(image_array):
    # Load image
    

    # Determine block size dynamically
    block_size = determine_block_size(image_array.shape)
    print(f"Using block size: {block_size}x{block_size}")

    # Divide image into blocks
    blocks, original_shape, padded_shape = divide_into_blocks(image_array, block_size)

    avg_intensity, variance, entropy = compute_image_characteristics(image_array)

    chen_keys = generate_chen_keys(image_array.size, avg_intensity, variance)
    lorenz_keys = generate_lorenz_keys(image_array.size, avg_intensity, entropy)
    logistic_keys = generate_logistic_keys(image_array.size, variance, entropy)
    hybrid_keys = (lorenz_keys + logistic_keys + chen_keys) % 256

    sbox = generate_dynamic_sbox(256, hybrid_keys)
    sbox_inv = np.argsort(sbox)

    iv = np.random.randint(0, 256, (block_size, block_size, 3), dtype=np.uint8)

    encrypted_blocks = []
    permutation_indices_list = []
    previous_block = iv  # Start with IV
    for block in blocks:
        encrypted_block, previous_block_flattened, permutation_indices = encrypt_block_cbc(
            block, hybrid_keys, sbox, previous_block
        )
        encrypted_blocks.append(encrypted_block)
        permutation_indices_list.append(permutation_indices)
        previous_block = previous_block_flattened.reshape(block.shape)  # Update to the current encrypted block for next iteration

    encrypted_image = merge_blocks(encrypted_blocks, original_shape, padded_shape, block_size)

    extracted_blocks, _, _ = divide_into_blocks(encrypted_image, block_size)

    decrypted_blocks = []
    previous_block = iv  # Start with IV again
    for i, block in enumerate(extracted_blocks):
        decrypted_block = decrypt_block_cbc(block, previous_block, permutation_indices_list[i], sbox_inv)
        decrypted_blocks.append(decrypted_block)
        previous_block = block  # Update to the current ciphertext block
    decrypted_image = merge_blocks(decrypted_blocks, original_shape, padded_shape, block_size)

    # plot_images(image_array, encrypted_image, decrypted_image, image_array.shape)
    return encrypted_image, decrypted_image
# if __name__ == "__main__":
#     # image_path = "C:/Users/20092/OneDrive/Documents/Bhumit/cyptography/imresizer-1732212939706.jpg"  # Replace with your image path
#     # main(image_path)
#     image = Image.open("C:/Users/20092/OneDrive/Documents/Bhumit/cyptography/cameraman.png").convert('RGB')  # Color image
#     image_array = np.array(image)
#     main(image_array)
    #main("C:/Users/20092/OneDrive/Documents/Bhumit/cyptography/peppers.png")
    
    # main("C:/Users/20092/OneDrive/Documents/Bhumit/cyptography/image.png")

In [105]:
def encrypt_image(image_array, block_size):
    """Encrypt an image and return the encrypted image, along with metadata."""
    iv = np.random.randint(0, 256, (block_size, block_size, image_array.shape[2]), dtype=np.uint8)
    blocks, original_shape, padded_shape = divide_into_blocks(image_array, block_size)

    encrypted_blocks = []
    permutation_indices_list = []
    characteristics_list = []
    previous_block = iv  # Initialization Vector (IV)

    for block in blocks:
        # Compute characteristics
        avg_intensity, variance, entropy = compute_image_characteristics(block)
        characteristics_list.append((avg_intensity, variance, entropy))

        # Generate hybrid keys and S-Box
        chen_keys = generate_chen_keys(block.size, avg_intensity, variance)
        lorenz_keys = generate_lorenz_keys(block.size, avg_intensity, entropy)
        logistic_keys = generate_logistic_keys(block.size, variance, entropy)
        hybrid_keys = (lorenz_keys + logistic_keys + chen_keys) % 256

        sbox = generate_dynamic_sbox(256, hybrid_keys)
        sbox_inv = np.argsort(sbox)

        # Encrypt block
        encrypted_block, previous_block_flattened, permutation_indices = encrypt_block_cbc(
            block, hybrid_keys, sbox, previous_block
        )
        encrypted_blocks.append(encrypted_block)
        permutation_indices_list.append(permutation_indices)
        previous_block = previous_block_flattened.reshape(block.shape)  # Update chaining block

    encrypted_image = merge_blocks(encrypted_blocks, original_shape, padded_shape, block_size)
    return encrypted_image, iv, permutation_indices_list, characteristics_list


In [106]:
def decrypt_image(encrypted_image, block_size, iv, permutation_indices_list, characteristics_list):
    """Decrypt an encrypted image using metadata."""
    blocks, original_shape, padded_shape = divide_into_blocks(encrypted_image, block_size)

    decrypted_blocks = []
    previous_block = iv  # Initialization Vector (IV)

    for i, block in enumerate(blocks):
        # Retrieve characteristics
        avg_intensity, variance, entropy = characteristics_list[i]

        # Generate hybrid keys and S-Box
        chen_keys = generate_chen_keys(block.size, avg_intensity, variance)
        lorenz_keys = generate_lorenz_keys(block.size, avg_intensity, entropy)
        logistic_keys = generate_logistic_keys(block.size, variance, entropy)
        hybrid_keys = (lorenz_keys + logistic_keys + chen_keys) % 256

        sbox = generate_dynamic_sbox(256, hybrid_keys)
        sbox_inv = np.argsort(sbox)

        # Decrypt block
        decrypted_block = decrypt_block_cbc(block, previous_block, permutation_indices_list[i], sbox_inv)
        decrypted_blocks.append(decrypted_block)
        previous_block = block  # Update chaining block

    return merge_blocks(decrypted_blocks, original_shape, padded_shape, block_size)


In [72]:
def calculate_npcr(original, modified):
    """Calculate NPCR (Number of Pixel Change Rate) between two images."""
    original = original.flatten()
    modified = modified.flatten()
    changed_pixels = np.sum(original != modified)
    total_pixels = original.size
    return (changed_pixels / total_pixels) * 100  # Percentage


In [99]:
def calculate_npcr_image(image_path):
    image = Image.open(image_path).convert('RGB')  # Color image
    image_array = np.array(image)
    encrypted_image,decrypted_image = main(image_array)
    modified_image = modify_single_pixel(image_array)
    modified_encrypted_image,modified_decrypted_image = main(modified_image)
    print(calculate_npcr(encrypted_image,modified_encrypted_image))

In [100]:
calculate_npcr_image(cameraman)
calculate_npcr_image(peppers)

(512, 512, 3)
Using block size: 64x64


C:\Users\20092\AppData\Local\Temp\ipykernel_31716\113127569.py:13: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  entropy = -np.sum(p * np.log2(p) for p in probabilities if p > 0)  # Entropy calculation


(512, 512, 3)
Using block size: 64x64
99.60416158040366
(384, 512, 3)
Using block size: 16x16
(384, 512, 3)
Using block size: 16x16
99.61276584201389


In [75]:
def calculate_uaci(original, modified):
    """Calculate UACI (Unified Average Changing Intensity) between two images."""
    original = original.flatten().astype(np.float32)
    modified = modified.flatten().astype(np.float32)
    uaci = np.sum(np.abs(original - modified)) / (255 * original.size)
    return uaci * 100  # Percentage


In [76]:
def calculate_uaci_image(image_path):
    image = Image.open(image_path).convert('RGB')  # Color image
    image_array = np.array(image)
    encrypted_image,decrypted_image = main(image_array)
    modified_image = modify_single_pixel(image_array)
    modified_encrypted_image,modified_decrypted_image = main(modified_image)
    print(calculate_uaci(encrypted_image,modified_encrypted_image))

In [96]:
calculate_uaci_image(cameraman)
calculate_uaci_image(peppers)

(512, 512, 3)
Using block size: 64x64


C:\Users\20092\AppData\Local\Temp\ipykernel_31716\113127569.py:13: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  entropy = -np.sum(p * np.log2(p) for p in probabilities if p > 0)  # Entropy calculation


(512, 512, 3)
Using block size: 64x64
33.46326242085376
(384, 512, 3)
Using block size: 16x16
(384, 512, 3)
Using block size: 16x16
33.501976528458606


In [119]:
def calculate_entropy(image):
    """Calculate the entropy of an image."""
    histogram, _ = np.histogram(image.flatten(), bins=256, range=(0, 255))
    probabilities = histogram / np.sum(histogram)
    entropy = -np.sum(p * np.log2(p) for p in probabilities if p > 0)
    return entropy


cameraman_encrypted_image,_ = main(cameraman_array)
print("cameraman",calculate_entropy(cameraman_array))
print(calculate_entropy(cameraman_encrypted_image))
peppers_encrypted_image,_ = main(peppers_array)
print("peppers",calculate_entropy(peppers_array))
print(calculate_entropy(peppers_encrypted_image))



(512, 512, 3)
Using block size: 64x64


C:\Users\20092\AppData\Local\Temp\ipykernel_31716\113127569.py:13: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  entropy = -np.sum(p * np.log2(p) for p in probabilities if p > 0)  # Entropy calculation
C:\Users\20092\AppData\Local\Temp\ipykernel_31716\2119213749.py:5: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  entropy = -np.sum(p * np.log2(p) for p in probabilities if p > 0)


cameraman 6.049670834469638
7.999755701594122
(384, 512, 3)
Using block size: 16x16
peppers 7.378534223569768
7.999666645646851


In [ ]:
def calculate_correlation(original):
    """Calculate the horizontal, vertical, and diagonal correlation of an image."""
    height, width = original.shape

    # Horizontal correlation
    horizontal = np.corrcoef(original[:, :-1].flatten(), original[:, 1:].flatten())[0, 1]

    # Vertical correlation
    vertical = np.corrcoef(original[:-1, :].flatten(), original[1:, :].flatten())[0, 1]

    # Diagonal correlation
    diagonal = np.corrcoef(original[:-1, :-1].flatten(), original[1:, 1:].flatten())[0, 1]

    return horizontal, vertical, diagonal


In [ ]:
def differential_analysis(original, encrypted_image, delta=1):
    """Perform differential analysis by changing one pixel and measuring NPCR and UACI."""
    modified_image = np.copy(original)
    modified_image[0, 0] = delta  # Change one pixel

    # Encrypt the modified image
    encrypted_modified = encrypt_image(modified_image, block_size)[0]  # Assuming `encrypt_image` is defined

    # Calculate NPCR and UACI
    npcr = calculate_npcr(encrypted_image, encrypted_modified)
    uaci = calculate_uaci(encrypted_image, encrypted_modified)

    return npcr, uaci


In [ ]:
def key_modification_analysis(image, block_size, characteristics_list, permutation_indices_list):
    """Evaluate how changing the key affects the encryption."""
    encrypted_image, iv, permutation_indices_list, characteristics_list = encrypt_image(image, block_size)
    
    # Modify key slightly (e.g., changing the first value)
    modified_permutation_indices = permutation_indices_list.copy()
    modified_permutation_indices[0] = (modified_permutation_indices[0] + 1) % 256

    # Re-encrypt with the modified key
    modified_encrypted_image = decrypt_image(encrypted_image, block_size, iv, modified_permutation_indices, characteristics_list)

    # Calculate NPCR and UACI for modified key
    npcr = calculate_npcr(encrypted_image, modified_encrypted_image)
    uaci = calculate_uaci(encrypted_image, modified_encrypted_image)

    return npcr, uaci


In [109]:
import math

def calculate_psnr(original, modified):
    """Calculate PSNR (Peak Signal-to-Noise Ratio) between two images."""
    mse = np.mean((original - modified) ** 2)
    if mse == 0:
        return 100  # Infinite PSNR
    max_pixel = 255.0
    return 20 * math.log10(max_pixel / math.sqrt(mse))


In [110]:
from skimage.metrics import structural_similarity as ssim

def calculate_ssim(original, modified):
    """Calculate SSIM (Structural Similarity Index) between two images."""
    return ssim(original, modified, multichannel=True)


In [126]:
from skimage.metrics import structural_similarity as ssim

def calculate_ms_ssim(original, modified):
    """Calculate MS-SSIM (Multi-Scale Structural Similarity Index) between two images."""
    return ssim(original, modified, multichannel=True, channel_axis =2,win_size=11, gaussian_weights=True, use_sample_covariance=False)


In [ ]:
print(calculate_ms_ssim(cameraman_array,cameraman_encrypted_image))
print(calculate_ms_ssim(peppers_array,peppers_encrypted_image))

0.008934633501083132

In [112]:
def average_effect(original, encrypted):
    """Calculate the average effect of encryption by measuring the pixel intensity differences."""
    diff = np.abs(original - encrypted)
    return np.mean(diff)


Processing: C:/Users/20092/OneDrive/Documents/Bhumit/cyptography\cameraman.png
(512, 512, 3)


C:\Users\20092\AppData\Local\Temp\ipykernel_31716\113127569.py:13: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  entropy = -np.sum(p * np.log2(p) for p in probabilities if p > 0)  # Entropy calculation


NPCR: 99.61%, UACI: 31.17%, PSNR: 8.39, , Entropy: 7.9998
Processing: C:/Users/20092/OneDrive/Documents/Bhumit/cyptography\imresizer-1732212939706.jpg
(50, 50, 3)


C:\Users\20092\AppData\Local\Temp\ipykernel_31716\2743621014.py:5: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  entropy = -np.sum(p * np.log2(p) for p in probabilities if p > 0)


IndexError: index 179 is out of bounds for axis 0 with size 3